In [29]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import xmljson

---
# XML format
---

XML имеют древовидную структуру. Что такое дерево? Это структура, которая имеет узлы и связи между ними. Самый верхнеуровневый узел называется корнем, а всё, что находится в самом низу, называется листьями. В примере выше корнем является menu, а листьями, например, price и weight. 

У menu есть дети (потомки) — это два узла dish, имеющие разный атрибут name.

Мы будем пользоваться модулем ElementTree, входящим в стандартный пакет xml. Этот модуль позволит нам ходить по дереву XML и смотреть, что находится в каждом его узле, начиная с корня и заканчивая листьями. Импортируем модуль:

<span style="color: orange; font-weight: bold; font-size:16pt">Чтение файла</span>

In [3]:
tree = ET.parse('menu.xml')
root = tree.getroot()

Здесь в переменной tree мы читаем всё дерево из XML файла. После этого в переменную root записываем корневой узел дерева tree. Давайте посмотрим, как выглядит root.

In [4]:
print(root)

<Element 'menu' at 0x000001E65609EE00>


Мы видим, что в корне находится 'menu'. Всё правильно, мы хотели увидеть именно это. Какой тип у этого объекта? Если мы вызовем встроенный в Python метод type и передадим ему root, то увидим, что тип — xml.etree.ElementTree.Element. Такой тип будет у любого узла в дереве.

Как посмотреть список детей (потомков) этого узла? С помощью метода getchildren():

In [5]:
root.getchildren()

<ipython-input-5-4340fffdc6c9>:1: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  root.getchildren()


[<Element 'dish' at 0x000001E65609EE50>,
 <Element 'dish' at 0x000001E65609EF90>]

Ранее мы видели, что у узлов могут быть параметры (атрибуты). Например, у узлов dish мы видели атрибут name. Мы можем обратиться к атрибутам объекта с помощью команды attrib. Здесь может возникнуть небольшая путаница, потому что мы говорим о двух разных определениях слова "атрибут" в нашем контексте:

Атрибут у тега (как name у dish).

Атрибут объекта (переменная класса).

В данном случае мы берем объект типа ElementTree.Element, у которого есть атрибут attrib. В этом атрибуте объекта хранится словарь с атрибутами данного узла.

Это значит, что если мы обратимся к attrib, то нам вернется словарь, в котором ключами будут названия атрибутов, а значениями — соответствующие значения. Чтобы стало яснее, давайте посмотрим на атрибуты у dish:

In [8]:
root[0].attrib

{'name': 'Кура'}

<span style="color: orange; font-weight: bold; font-size:16pt">Чтение значений узлов</span>

В узлах XML часто хранятся количественные показатели. Эти показатели хранятся в виде текста, и прочитать их можно, обратившись к атрибуту text у соответствующего объекта типа ElementTree.Element. Например, возьмем узел price:

In [11]:
root[0][0]

<Element 'price' at 0x000001E65609EEA0>

In [12]:
root[0][0].text

'40'

Таким образом можно пройтись по всему дереву XML и прочитать значения в листьях. Если файл имеет четкую структуру, его во многих случаях можно превратить в таблицу и работать с ней уже в pandas. Об этом мы поговорим позже.

Заметьте, что значение '40' представлено строкой. Все значения в XML хранятся как строки, поэтому преобразовывать их к нужному типу вам нужно самим. Например, в данном случае можно обернуть значение стоимости в int() или float().

Если вы хотите прочитать название тега для конкретного узла, можно использовать атрибут tag.

In [16]:
root[0][1]

<Element 'weight' at 0x000001E65609EEF0>

<span style="color: orange; font-weight: bold; font-size:16pt">Использование циклов</span>

Так как к узлам можно обращаться как к спискам, можно использовать циклы для итерации по детям узлов.

Например, если мы хотим напечатать все значения в листьях, то есть название блюда, название параметра этого блюда и значение параметра, то поможет следующий код:

In [20]:
for elem in root:
    for subelem in elem:
        print(elem.attrib['name'],subelem.tag,subelem.text)
        print()

Кура price 40

Кура weight 300

Кура class Мясо

Греча price 20

Греча weight 200

Греча class Крупа



In [21]:
len(root[0])

3

---
# 12.12 DataFrame и JSON из XML
---

**Превращение XML в pd.DataFrame**

In [26]:
df_index = ['name','price','weight','class']
df = pd.DataFrame(columns=df_index)

for elem in root:
    elements = [elem.get('name'),elem[0].text,elem[1].text,elem[2].text]
    df = df.append(pd.Series(elements,index = df_index),ignore_index = True)

In [27]:
df

,name,price,weight,class
0,Кура,40,300,Мясо
1,Греча,20,200,Крупа


Что здесь произошло?

1. Мы задали названия столбцов в новой таблице и создали пустой DataFrame.

2.Затем мы прошлись по всем детям из корня нашего дерева и составили строки (pd.Series), состоящие из содержимого этих элементов: взяли атрибут name у узла и значения всех его детей, которые содержат нужные нам данные. Можно заметить, что для получения названия продукта мы использовали метод get(). Это можно делать, так как атрибуты узла хранятся в виде словаря и get() — один из способов получить значения словаря, зная соответствующие ключи. 

3. После этого мы добавили новую строку в DataFrame с помощью метода append().

Таким образом мы можем трансформировать XML в DataFrame. К сожалению, стандартных средств для этого превращения не существует, но если XML файл содержит чёткую структуру, сделать это средствами Python очень просто, как вы увидели выше.

<span style="color: orange; font-weight: bold; font-size:16pt">Превращение XML в JSON</span>

Ранее вы могли обратить внимание, что XML похож на JSON. Для Python есть несколько сторонних библиотек, которые позволяют трансформировать XML в JSON. Одна из таких библиотек (пакетов) — xmljson. Есть несколько принятых соглашений (conventions) по превращению XML в JSON. Рассмотрим простой пример (пример был взят отсюда), прежде чем поработать с нашими данными. Допустим, наши данные выглядят так:

In [30]:
xmljson.parker.data(root)

OrderedDict([('dish',
              [OrderedDict([('price', 40),
                            ('weight', 300),
                            ('class', 'Мясо')]),
               OrderedDict([('price', 20),
                            ('weight', 200),
                            ('class', 'Крупа')])])])

In [31]:
xmljson.gdata.data(root)  

OrderedDict([('menu',
              OrderedDict([('dish',
                            [OrderedDict([('name', 'Кура'),
                                          ('price', OrderedDict([('$t', 40)])),
                                          ('weight',
                                           OrderedDict([('$t', 300)])),
                                          ('class',
                                           OrderedDict([('$t', 'Мясо')]))]),
                             OrderedDict([('name', 'Греча'),
                                          ('price', OrderedDict([('$t', 20)])),
                                          ('weight',
                                           OrderedDict([('$t', 200)])),
                                          ('class',
                                           OrderedDict([('$t',
                                                         'Крупа')]))])])]))])

Как превратить JSON обратно в XML? Использовать метод etree() у класса-соглашения:

In [32]:
parker_json = xmljson.parker.data(root)  
back_to_xml = xmljson.parker.etree(parker_json) 

В переменной back_to_xml будет храниться наш изначальный XML-файл в формате списка, содержащего дочерние узлы корня дерева: